# Dec 2021
A binary classifier to predict whether to pit or not

## Parth Khurana

## importing pandas and numpy <a class="anchor" id="Section1"></a>

In [31]:
import pandas as pd
import numpy as np

### Import Data <a class="anchor" id="section_1_1"></a>

In [32]:
laps_master = pd.read_csv('PitStop_Dataset/lap_times.csv')
races_master = pd.read_csv('PitStop_Dataset/races.csv')
quali_master = pd.read_csv('PitStop_Dataset/qualifying.csv')
drivers_master = pd.read_csv('PitStop_Dataset/drivers.csv')
constructors_master = pd.read_csv('PitStop_Dataset/constructors.csv')
results_master = pd.read_csv('PitStop_Dataset/results.csv')
circuits_master = pd.read_csv('PitStop_Dataset/circuits.csv')
pits_master = pd.read_csv('PitStop_Dataset/pit_stops.csv')

In [33]:
pits_master

,raceId,driverId,stop,lap,time,duration,milliseconds
0,841,153,1,1,17:05:23,26.898,26898
1,841,30,1,1,17:05:52,25.021,25021
2,841,17,1,11,17:20:48,23.426,23426
3,841,4,1,12,17:22:34,23.251,23251
4,841,13,1,13,17:24:10,23.842,23842
...,...,...,...,...,...,...,...
7868,1043,1,2,52,14:22:55,30.925,30925
7869,1043,20,2,51,14:22:57,29.934,29934
7870,1043,840,3,52,14:25:09,36.630,36630
7871,1043,846,2,53,14:25:29,30.203,30203


### Pit Stop Table Transformation <a class="anchor" id="section_1_2"></a>

Create new data frame with a list of laps when a pit stop was occuring for each driver, for each race

In [34]:
pits_df_new = pits_master.groupby(['raceId', 'driverId'])['lap'].apply(list).reset_index(name='laps_when_pitstop')
pits_df_new

,raceId,driverId,laps_when_pitstop
0,841,1,"[16, 36]"
1,841,2,"[15, 30]"
2,841,3,[16]
3,841,4,"[12, 27, 42]"
4,841,5,[17]
...,...,...,...
3813,1043,844,[13]
3814,1043,846,"[13, 53]"
3815,1043,847,[11]
3816,1043,848,"[14, 59]"


#### Preview the lap times table
Let's take a look at a random race, and random driver, and see how the lap times look .. just to better understand what transformation needs to be done on the data

In [35]:
laps_master[laps_master.raceId == 841][laps_master.driverId == 17]

/var/folders/px/v7yqgpyd40scn7866wnfg9yw0000gn/T/ipykernel_1125/1654452511.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  laps_master[laps_master.raceId == 841][laps_master.driverId == 17]


,raceId,driverId,lap,position,time,milliseconds
116,841,17,1,3,1:41.467,101467
117,841,17,2,3,1:33.725,93725
118,841,17,3,3,1:33.208,93208
119,841,17,4,3,1:32.933,92933
120,841,17,5,3,1:32.899,92899
121,841,17,6,3,1:32.916,92916
122,841,17,7,3,1:32.900,92900
123,841,17,8,3,1:33.262,93262
124,841,17,9,3,1:33.330,93330
125,841,17,10,3,1:34.248,94248


### Lap Times Table Transformation <a class="anchor" id="section_1_3"></a>

Create a new data frame containing a list of all the lap times in one row, for an entire race, for each driver

In [36]:
laps_df_new = laps_master.groupby(['raceId', 'driverId'])['milliseconds'].apply(list).reset_index(name='race_lap_times')
laps_df_new

,raceId,driverId,race_lap_times
0,1,1,"[109088, 93740, 91600, 91067, 92129, 90469, 89..."
1,1,2,"[157754, 97640, 91697, 91867, 92234, 92036, 91..."
2,1,3,"[102659, 92091, 91148, 89660, 89468, 89643, 90..."
3,1,4,"[109367, 94983, 92175, 93114, 91831, 91556, 91..."
4,1,6,"[104899, 92734, 92684, 90952, 90152, 89923, 90..."
...,...,...,...
9208,1043,844,"[87074, 82058, 81592, 80850, 80815, 80638, 807..."
9209,1043,846,"[89178, 82509, 81694, 81372, 81115, 81994, 808..."
9210,1043,847,"[91472, 83350, 81994, 81787, 81551, 81349, 811..."
9211,1043,848,"[87853, 82182, 81611, 80812, 80772, 80680, 811..."


### Left Join New Pit-Stop Table with New Lap-Times Table <a class="anchor" id="section_1_4"></a>

In [37]:
merged = pd.merge(pits_df_new, laps_df_new, on=['raceId', 'driverId'], how='left')
merged

,raceId,driverId,laps_when_pitstop,race_lap_times
0,841,1,"[16, 36]","[100573, 93774, 92900, 92582, 92471, 92434, 92..."
1,841,2,"[15, 30]","[112091, 96505, 95067, 95662, 95405, 95662, 95..."
2,841,3,[16],"[105340, 95808, 95436, 95810, 96837, 94856, 94..."
3,841,4,"[12, 27, 42]","[106144, 96233, 94726, 95355, 94400, 93278, 92..."
4,841,5,[17],"[112883, 98252, 97378, 96628, 97441, 96512, 96..."
...,...,...,...,...
3813,1043,844,[13],"[87074, 82058, 81592, 80850, 80815, 80638, 807..."
3814,1043,846,"[13, 53]","[89178, 82509, 81694, 81372, 81115, 81994, 808..."
3815,1043,847,[11],"[91472, 83350, 81994, 81787, 81551, 81349, 811..."
3816,1043,848,"[14, 59]","[87853, 82182, 81611, 80812, 80772, 80680, 811..."


### Lap Times Before Pit-Stop Sequence Partitioning <a class="anchor" id="section_1_5"></a>

In [38]:
def partition_lapTime_into_sequences(pitStop_laps, race_lapTimes):
    # NOTE: no need to return the last stint, since it is not followed by a pit stop... 
    #       only return sequence of lap times that are followed by a pit stop

    # returns: list of lap time sequences (which as lists) ... so list of lists
    
    # remove pit stops from first lap... those occur because of a collision, so they should not be looked at when predicting the end of the stint
    if 1 in pitStop_laps:
        pitStop_laps = pitStop_laps[1:]                        # remove first lap pit stop, as it was not a regular, planned one
        race_lapTimes = race_lapTimes[1:]                      # remove the first lap time, since the stint was "corrupted" by the emergency pitstop 
        pitStop_laps[:] = [x - 1 for x in pitStop_laps]        # subtract one lap from the pit-stop lap count, to account for the first lap being removed
    
    if len(pitStop_laps) < 1:
        return np.nan    # no real stints have occured. Pitted on lap 1, then never pitted again during the race.
    
    sequences = []
    prev_pit = pitStop_laps[0]
    
    if len(pitStop_laps) == 1:   # if the race is a one-stop race 
        sequences.append(race_lapTimes[:prev_pit-1])   # the off-by-one accounts for not taking into consideration the lap with the pit-stop as part of the sequence
    else:                      # multi-stop race as
        
        for current_pit in pitStop_laps:
            if current_pit == prev_pit:           # this is only true when prev_pit = pitStop_laps[0]
                sequences.append(race_lapTimes[:current_pit-1])         # create first stint
                # the off-by-one accounts for not taking into consideration the lap with the pit-stop as part of the sequence
            else:
                sequences.append(race_lapTimes[prev_pit:current_pit-1]) # create next sequence from (prev-pit-lap, current_pit-lap)
                prev_pit = current_pit                             # update pointer to previous pit ... this will be needed for the next pit
    return sequences

### Sequencing Function Test cases

In [39]:
sample_input_pits = merged.iloc[13, :].laps_when_pitstop
sample_input_lapTimes = merged.iloc[13, :].race_lap_times

print("input pits: ", sample_input_pits)
print("input laps: ", sample_input_lapTimes)

print("output: ", partition_lapTime_into_sequences(sample_input_pits, sample_input_lapTimes))

input pits:  [1, 17]
input laps:  [161428, 103027, 96602, 96062, 95839, 95581, 95794, 97682, 96764, 98516, 97483, 95422, 95319, 97854, 98276, 96991, 117213, 102267, 95367]
output:  [[103027, 96602, 96062, 95839, 95581, 95794, 97682, 96764, 98516, 97483, 95422, 95319, 97854, 98276, 96991]]


To DO: write test cases

### Get Lap Times of Final Stint (as a non-pit-stint)

In [40]:
def get_last_stint_lap_times(pitStop_laps, race_lapTimes):
    # returns the last stint's lap times, since it is not followed by a pit stop .. so it is non-pit-stop stint 

    last_pit = pitStop_laps[-1]
    return race_lapTimes[last_pit:]

### Test get_last_stint_lap_times function 

In [41]:
sample_input_pits = merged.iloc[13, :].laps_when_pitstop
sample_input_lapTimes = merged.iloc[13, :].race_lap_times

print("input pits: ", sample_input_pits)
print("input laps: ", sample_input_lapTimes)

print("output: ", get_last_stint_lap_times(sample_input_pits, sample_input_lapTimes))

input pits:  [1, 17]
input laps:  [161428, 103027, 96602, 96062, 95839, 95581, 95794, 97682, 96764, 98516, 97483, 95422, 95319, 97854, 98276, 96991, 117213, 102267, 95367]
output:  [102267, 95367]


In [42]:
sample_input_pits = merged.iloc[1, :].laps_when_pitstop
sample_input_lapTimes = merged.iloc[1, :].race_lap_times

print("input pits: ", sample_input_pits)
print("input laps: ", sample_input_lapTimes)

print("output: ", get_last_stint_lap_times(sample_input_pits, sample_input_lapTimes))

input pits:  [15, 30]
input laps:  [112091, 96505, 95067, 95662, 95405, 95662, 95698, 95363, 95895, 95038, 94838, 95039, 96546, 95592, 114318, 99684, 93707, 93539, 93421, 93822, 94361, 93889, 94735, 93587, 94694, 93545, 93263, 92848, 93282, 114826, 99960, 92876, 92552, 93071, 95177, 92892, 92765, 92993, 94306, 94974, 93026, 92807, 92377, 92619, 93120, 92898, 93102, 95082, 94204, 95702, 93801, 95493, 95589, 95243, 96404, 93858, 95088]
output:  [99960, 92876, 92552, 93071, 95177, 92892, 92765, 92993, 94306, 94974, 93026, 92807, 92377, 92619, 93120, 92898, 93102, 95082, 94204, 95702, 93801, 95493, 95589, 95243, 96404, 93858, 95088]


### Applying the sequence partitioning function to the merged column

In [43]:
merged['stints'] = merged.apply(lambda x: partition_lapTime_into_sequences(x.laps_when_pitstop, x.race_lap_times), axis=1)
merged['last_stint'] = merged.apply(lambda x: get_last_stint_lap_times(x.laps_when_pitstop, x.race_lap_times), axis=1)
merged

,raceId,driverId,laps_when_pitstop,race_lap_times,stints,last_stint
0,841,1,"[16, 36]","[100573, 93774, 92900, 92582, 92471, 92434, 92...","[[100573, 93774, 92900, 92582, 92471, 92434, 9...","[99548, 91013, 90338, 91200, 90314, 90421, 910..."
1,841,2,"[15, 30]","[112091, 96505, 95067, 95662, 95405, 95662, 95...","[[112091, 96505, 95067, 95662, 95405, 95662, 9...","[99960, 92876, 92552, 93071, 95177, 92892, 927..."
2,841,3,[16],"[105340, 95808, 95436, 95810, 96837, 94856, 94...","[[105340, 95808, 95436, 95810, 96837, 94856, 9...","[102954, 94939, 94011, 93614, 93503, 94600]"
3,841,4,"[12, 27, 42]","[106144, 96233, 94726, 95355, 94400, 93278, 92...","[[106144, 96233, 94726, 95355, 94400, 93278, 9...","[97745, 90029, 89608, 89840, 89837, 89597, 894..."
4,841,5,[17],"[112883, 98252, 97378, 96628, 97441, 96512, 96...","[[112883, 98252, 97378, 96628, 97441, 96512, 9...","[101782, 94918]"
...,...,...,...,...,...,...
3813,1043,844,[13],"[87074, 82058, 81592, 80850, 80815, 80638, 807...","[[87074, 82058, 81592, 80850, 80815, 80638, 80...","[104016, 81189, 80770, 80186, 80005, 80201, 79..."
3814,1043,846,"[13, 53]","[89178, 82509, 81694, 81372, 81115, 81994, 808...","[[89178, 82509, 81694, 81372, 81115, 81994, 80...","[130065, 133405, 130208, 123885, 81717, 79417,..."
3815,1043,847,[11],"[91472, 83350, 81994, 81787, 81551, 81349, 811...","[[91472, 83350, 81994, 81787, 81551, 81349, 81...","[102788, 79705, 80345, 80466, 80243, 80426, 80..."
3816,1043,848,"[14, 59]","[87853, 82182, 81611, 80812, 80772, 80680, 811...","[[87853, 82182, 81611, 80812, 80772, 80680, 81...","[102008, 76495, 80248, 76177]"


Check if there are any missing stints

In [44]:
merged.isnull().sum()

raceId                0
driverId              0
laps_when_pitstop     0
race_lap_times        0
stints               36
last_stint            0
dtype: int64

There are some missing values based on the sequence partitioning transformation that we have just applied. Let's see where they are.

In [45]:
merged[merged.isnull().any(axis=1)]

,raceId,driverId,laps_when_pitstop,race_lap_times,stints,last_stint
366,857,10,[1],"[116549, 129008]",NaN,[129008]
554,865,155,[1],"[135620, 104521, 89493, 85480, 86884]",NaN,"[104521, 89493, 85480, 86884]"
626,868,814,[1],"[139266, 156909]",NaN,[156909]
694,871,813,[1],"[171174, 179089, 182854, 176181]",NaN,"[179089, 182854, 176181]"
813,877,3,[1],"[120945, 138242, 109572, 109340, 109413, 11022...",NaN,"[138242, 109572, 109340, 109413, 110220, 110227]"
1173,895,1,[1],"[151257, 123558, 101202, 101427, 101467, 10136...",NaN,"[123558, 101202, 101427, 101467, 101363, 112137]"
1311,901,824,[1],"[188057, 164320, 111480, 111473, 111603, 11168...",NaN,"[164320, 111480, 111473, 111603, 111686, 11257..."
1385,905,16,[1],"[145066, 104523, 107029, 88976, 85705, 84913, ...",NaN,"[104523, 107029, 88976, 85705, 84913, 85215, 8..."
1582,915,3,[1],"[112669, 129911, 105571, 105340, 106992, 10496...",NaN,"[129911, 105571, 105340, 106992, 104962, 10533..."
1609,916,807,[1],"[116291, 173995, 151249, 144665, 109172, 10810...",NaN,"[173995, 151249, 144665, 109172, 108108, 10650..."


As I have thought, all cases are just races when there was only one pit stop, on lap 1, so for the scope of this end-of-stint classifier we can safely remove theses cases, as they do not affect the task at hand

In [46]:
merged = merged.dropna()
merged

,raceId,driverId,laps_when_pitstop,race_lap_times,stints,last_stint
0,841,1,"[16, 36]","[100573, 93774, 92900, 92582, 92471, 92434, 92...","[[100573, 93774, 92900, 92582, 92471, 92434, 9...","[99548, 91013, 90338, 91200, 90314, 90421, 910..."
1,841,2,"[15, 30]","[112091, 96505, 95067, 95662, 95405, 95662, 95...","[[112091, 96505, 95067, 95662, 95405, 95662, 9...","[99960, 92876, 92552, 93071, 95177, 92892, 927..."
2,841,3,[16],"[105340, 95808, 95436, 95810, 96837, 94856, 94...","[[105340, 95808, 95436, 95810, 96837, 94856, 9...","[102954, 94939, 94011, 93614, 93503, 94600]"
3,841,4,"[12, 27, 42]","[106144, 96233, 94726, 95355, 94400, 93278, 92...","[[106144, 96233, 94726, 95355, 94400, 93278, 9...","[97745, 90029, 89608, 89840, 89837, 89597, 894..."
4,841,5,[17],"[112883, 98252, 97378, 96628, 97441, 96512, 96...","[[112883, 98252, 97378, 96628, 97441, 96512, 9...","[101782, 94918]"
...,...,...,...,...,...,...
3813,1043,844,[13],"[87074, 82058, 81592, 80850, 80815, 80638, 807...","[[87074, 82058, 81592, 80850, 80815, 80638, 80...","[104016, 81189, 80770, 80186, 80005, 80201, 79..."
3814,1043,846,"[13, 53]","[89178, 82509, 81694, 81372, 81115, 81994, 808...","[[89178, 82509, 81694, 81372, 81115, 81994, 80...","[130065, 133405, 130208, 123885, 81717, 79417,..."
3815,1043,847,[11],"[91472, 83350, 81994, 81787, 81551, 81349, 811...","[[91472, 83350, 81994, 81787, 81551, 81349, 81...","[102788, 79705, 80345, 80466, 80243, 80426, 80..."
3816,1043,848,"[14, 59]","[87853, 82182, 81611, 80812, 80772, 80680, 811...","[[87853, 82182, 81611, 80812, 80772, 80680, 81...","[102008, 76495, 80248, 76177]"


In [47]:
end_of_stint_sequences = merged['stints']
end_of_stint_sequences[0]

[[100573,
  93774,
  92900,
  92582,
  92471,
  92434,
  92447,
  92310,
  92612,
  93121,
  92737,
  92795,
  92475,
  92984,
  93186],
 [99566,
  91823,
  91400,
  91194,
  91212,
  91524,
  91414,
  91564,
  91825,
  91419,
  91510,
  92267,
  91826,
  91149,
  91453,
  94258,
  92133,
  92016,
  92427]]

In [48]:
last_stint_sequences = merged['last_stint']
last_stint_sequences[0]

[99548,
 91013,
 90338,
 91200,
 90314,
 90421,
 91085,
 91175,
 90512,
 90508,
 90637,
 90785,
 90753,
 91010,
 91312,
 91629,
 91470,
 91789,
 91766,
 91813,
 92184,
 94576]

We need to flatten the structure of the data. We need a list of lists, not a Pandas Series of lists of lists

In [49]:
temp = end_of_stint_sequences.tolist()  # lists of lists of lists
print("Before:", temp[0:3])
print()

Before: [[[100573, 93774, 92900, 92582, 92471, 92434, 92447, 92310, 92612, 93121, 92737, 92795, 92475, 92984, 93186], [99566, 91823, 91400, 91194, 91212, 91524, 91414, 91564, 91825, 91419, 91510, 92267, 91826, 91149, 91453, 94258, 92133, 92016, 92427]], [[112091, 96505, 95067, 95662, 95405, 95662, 95698, 95363, 95895, 95038, 94838, 95039, 96546, 95592], [99684, 93707, 93539, 93421, 93822, 94361, 93889, 94735, 93587, 94694, 93545, 93263, 92848, 93282]], [[105340, 95808, 95436, 95810, 96837, 94856, 94619, 94624, 94691, 94021, 94097, 94862, 94407, 94135, 94473]]]



In [50]:
# Use list.extend() to convert a a 3D list to a 2D lists
end_of_stint_sequences = []
for elem in temp:
    end_of_stint_sequences.extend(elem)       # this will make it lists of lists

print("After:", end_of_stint_sequences[0:3])
print("Sample Size = ", len(end_of_stint_sequences))

After: [[100573, 93774, 92900, 92582, 92471, 92434, 92447, 92310, 92612, 93121, 92737, 92795, 92475, 92984, 93186], [99566, 91823, 91400, 91194, 91212, 91524, 91414, 91564, 91825, 91419, 91510, 92267, 91826, 91149, 91453, 94258, 92133, 92016, 92427], [112091, 96505, 95067, 95662, 95405, 95662, 95698, 95363, 95895, 95038, 94838, 95039, 96546, 95592]]
Sample Size =  7612


### Generate not end of stint sequences --- this method did not work

In [51]:
def remove_lastN_elements(arr, N):
    return arr[:-N]

print(end_of_stint_sequences[0])
print()
print(remove_lastN_elements(end_of_stint_sequences[0], 2))

[100573, 93774, 92900, 92582, 92471, 92434, 92447, 92310, 92612, 93121, 92737, 92795, 92475, 92984, 93186]

[100573, 93774, 92900, 92582, 92471, 92434, 92447, 92310, 92612, 93121, 92737, 92795, 92475]


In [52]:
NOT_end_of_stint_sequences = []

# N needs to be experimented with. Initially I chose N=1, N=2, and N=4
for lst in end_of_stint_sequences:
    temp_list = remove_lastN_elements(lst, N = 1)        # remove last lap from each stint
    NOT_end_of_stint_sequences.append(temp_list)
    
    temp_list = remove_lastN_elements(lst, N = 2)        # remove last 2 laps from each stint
    NOT_end_of_stint_sequences.append(temp_list)
    
    temp_list = remove_lastN_elements(lst, N = 4)        # remove last 4 laps from each stint
    NOT_end_of_stint_sequences.append(temp_list)

In [53]:
#print(len(NOT_end_of_stint_sequences))
print(len(end_of_stint_sequences))

7612


RESULT = 3:1 ratio between not-end-of-stint and end-os-stint data

Let's create the labels:

In [54]:
end_of_stint_labels = [1] * len(end_of_stint_sequences)
NOT_end_of_stint_labels = [0] * len(NOT_end_of_stint_sequences)

### Creating instances of not an end of tyre stint

In [55]:
NOT_end_of_stint_sequences = last_stint_sequences.tolist() 
NOT_end_of_stint_labels = [0] * len(NOT_end_of_stint_sequences)
end_of_stint_labels = [1] * len(end_of_stint_sequences)
print("Labels:")
print(len(NOT_end_of_stint_labels))
print(len(end_of_stint_labels))
print("\nSequences:")
print(len(NOT_end_of_stint_sequences))
print(len(end_of_stint_sequences))

Labels:
3782
7612

Sequences:
3782
7612


In [56]:
stint_sequences = end_of_stint_sequences + NOT_end_of_stint_sequences
stint_labels = end_of_stint_labels + NOT_end_of_stint_labels

print(len(stint_sequences))
print(len(stint_labels))

11394
11394


### Test train split

In [57]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(stint_sequences, stint_labels, test_size=0.20, random_state=7)

print("Train set:", len(X_train))
print("Test set:", len(X_test))
print("Train labels:", len(y_train))
print("Test labels:", len(y_test))

Train set: 9115
Test set: 2279
Train labels: 9115
Test labels: 2279


### Pad Input Sequences 

In [58]:
# find out what's the longest stint in our data set
#max_stint_length = max(map(len, end_of_stint_sequences))
#print("Max stint-length =", max_stint_length)

max_stint_length = 30 

Wrap every list into numpy arrays, so Keras can process the input

In [59]:
from keras.preprocessing import sequence

X_train = sequence.pad_sequences(X_train, maxlen=max_stint_length, padding="pre", truncating='pre')
X_test = sequence.pad_sequences(X_test, maxlen=max_stint_length, padding="pre", truncating='pre'

X_train = np.array(X_train)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

X_test = np.array(X_test)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

y_train = np.array(y_train)
y_train = y_train.reshape(y_train.shape[0], 1)

y_test = np.array(y_test)
y_test = y_test.reshape(y_test.shape[0], 1)

SyntaxError: invalid syntax (3959967485.py, line 6)

testing the ratio of 0 to 1 in the train and test set

In [ ]:
unique, frequency = np.unique(y_test,  
                              return_counts = True) 
# print unique values array 
print("Unique Values:",  
      unique) 
  
# print frequency array 
print("Frequency Values:", 
      frequency)

In [ ]:
unique, frequency = np.unique(y_train,  
                              return_counts = True) 
# print unique values array 
print("Unique Values:",  
      unique) 
  
# print frequency array 
print("Frequency Values:", 
      frequency)

### Approach 1: Simple LSTM for Sequence Classification

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# fix random seed for reproducibility
np.random.seed(7)

In [ ]:
model = Sequential()
model.add(LSTM(100, input_shape=(max_stint_length, 1)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=9, batch_size=128)

In [ ]:
model = Sequential()
model.add(LSTM(64, input_shape=(max_stint_length, 1), return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=9, batch_size=128)

### LSTM Layers (Time distributed LTSM)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, TimeDistributed, LSTM, Dropout

# fix random seed for reproducibility
np.random.seed(7)

In [ ]:
model = Sequential()
model.add(LSTM(512, input_shape=(max_stint_length, 1), return_sequences=True))
#model.add(Dropout(0.3))
model.add(LSTM(256, return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(LSTM(64, return_sequences=True))
model.add(TimeDistributed(Dense(1, activation='sigmoid')))


model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128)

### using an 8 layer CNN network

In [ ]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

model = Sequential()
model.add(Conv1D(filters=256, kernel_size=3, padding='same', activation='relu', input_shape=(max_stint_length, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=128)